In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [8, 8]
sns.set_theme(style="whitegrid")
sns.color_palette("rocket", as_cmap=True)
sns.set_palette("pastel")
#hide all warnings
import warnings
warnings.filterwarnings('ignore')

In [30]:
data_dir = "../data/"
plots_dir = "../plots/"

In [31]:
reviews = pd.read_csv(data_dir+"gta-reviews.csv")

In [32]:
reviews

,title,score,critic,summary,score_critic,year
0,Grand Theft Auto: Liberty City Stories,88,Gamer 2.0,"On one hand, it is watered-down, it is not as ...",83.0,24-Oct-05
1,Grand Theft Auto: San Andreas,93,Gamer's Hell,"An absolute blast to play, if you can forgive ...",78.0,07-Jun-05
2,Grand Theft Auto: Chinatown Wars,93,IGN AU,For Nintendo-only gamers out there – and these...,93.0,17-Mar-09
3,Grand Theft Auto: San Andreas,95,NaN,Bigger and richer in every way - a near bottom...,88.0,26-Oct-04
4,Grand Theft Auto: Vice City,95,Thunderbolt,"It has the right blend of intense gameplay, an...",90.0,27-Oct-02
...,...,...,...,...,...,...
823,Grand Theft Auto IV: The Ballad of Gay Tony,89,NaN,Slightly less satisfying than last time around...,90.0,29-Oct-09
824,Grand Theft Auto: Vice City 10th Anniversary E...,80,NaN,If you aren't grinning by mission two then the...,82.0,06-Dec-12
825,Grand Theft Auto: Liberty City Stories,78,NaN,"Spectacularly good value, despite some of the ...",90.0,06-Jun-06
826,Grand Theft Auto IV: The Lost and Damned,88,Vandal,A dark and sordid revisit to Liberty City full...,90.0,13-Apr-10


In [33]:
reviews['date'] =  pd.to_datetime(reviews['year'])


In [34]:
reviews['date'].describe()

count                     828
unique                     15
top       2005-06-07 00:00:00
freq                      105
first     2002-10-27 00:00:00
last      2012-12-06 00:00:00
Name: date, dtype: object

We have reviews of Grand Theft Auto series launched between 2002-2012

In [ ]:
#processing review data and then calculating the sentiment score 
